In [121]:
import ROOT
import ROOT as rt

In [50]:
year = 2018
# njet = 0
# for njet in [0,1,2]:
for njet in [0]:
    file = ROOT.TFile(f"{year}_njet{njet}.root", "READ")
    save_path = "./plots"
    workspace = file.Get("zpt_Workspace")
    # target_nbins = 50
    # for target_nbins in [25, 50, 100, 250]:
    print(f"{year} njet{njet}------------------------------------------------------------------------------------------------------")
    for target_nbins in [50, 100]:
        hist_data = workspace.obj("hist_data").Clone("hist_data_clone")
        hist_dy = workspace.obj("hist_dy").Clone("hist_dy_clone")
        orig_nbins = hist_data.GetNbinsX()
        rebin_coeff = int(orig_nbins/target_nbins)
        print(f"rebin_coeff: {rebin_coeff}")
        hist_data = hist_data.Rebin(rebin_coeff, "rebinned hist_data") 
        hist_dy = hist_dy.Rebin(rebin_coeff, "rebinned hist_dy") 
        
        hist_SF = hist_data.Clone("hist_SF")
        hist_SF.Divide(hist_dy)
        
        
        
        # Draw the histogram and fit
        canvas = ROOT.TCanvas("canvas", f"{target_nbins} bins Data and DY", 800, 600)
        hist_data.SetLineColor(ROOT.kRed)
        hist_dy.SetLineColor(ROOT.kBlue)
        # Change the plot title
        hist_data.SetTitle(f"njet {njet} {target_nbins} bins Data and DY")
        hist_data.Draw()
        
        hist_dy.Draw("SAME")
        # Add a legend
        legend = ROOT.TLegend(0.7, 0.7, 0.9, 0.9)  # Legend coordinates (x1, y1, x2, y2)
        legend.AddEntry(hist_data, "Data", "l")  # "l" means line style
        legend.AddEntry(hist_dy, "DY", "l")
        legend.Draw()
        canvas.SetLogy(1)
        canvas.Update()
        # canvas.SaveAs(f"{save_path}/{year}_njet{njet}_{target_nbins}Bins_DataDy_Hist.png")
        # canvas.SaveAs(f"{save_path}/{year}_njet{njet}_{target_nbins}Bins_DataDy_Hist.pdf")
        
        canvas = ROOT.TCanvas("canvas", f"{target_nbins} bins SF hist", 800, 600)
        hist_SF.SetTitle(f"njet {njet} {target_nbins} bins SF")
        hist_SF.SetMinimum(0.5)  # Set the lower bound of the Y-axis
        hist_SF.SetMaximum(4)  # Set the upper bound of the Y-axis
        hist_SF.Draw()
        
        canvas.Update()
        # canvas.SaveAs(f"{save_path}/{year}_njet{njet}_{target_nbins}Bins_SF_Hist.png")
        # canvas.SaveAs(f"{save_path}/{year}_njet{njet}_{target_nbins}Bins_SF_Hist.pdf")

        
        dimuon_pt = ROOT.RooRealVar("dimuon_pt", "Dimuon pT", 0, 200)

        # Convert the TH1F histogram to a RooDataHist
        roo_hist_SF = ROOT.RooDataHist("roo_hist", "RooFit Histogram", ROOT.RooArgList(dimuon_pt), hist_SF)
        
        # Print information about the RooDataHist
        roo_hist_SF.Print()
        

        chi2_dof_l = []
        for order in range(3,16):
            hist_SF.SetTitle(f"{order} order poly, njet {njet}, {target_nbins} bins SF")
            polynomial_expr = " + ".join([f"[{i}]*x**{i}" for i in range(order+1)])
            polynomial_func = ROOT.TF1(f"poly{order}", polynomial_expr, -5, 5)
            # Define the TF1 function with the generated expression
            fit_func = polynomial_func
            _ = hist_SF.Fit(fit_func, "S")
            _ = hist_SF.Fit(fit_func, "S")
            fit_result = hist_SF.Fit(fit_func, "S")

            chi2 = fit_func.GetChisquare()
            ndf = fit_func.GetNDF()

            # Calculate chi2/dof
            chi2_dof = chi2 / ndf if ndf > 0 else float("inf")
            
            # Print fit results
            # print(f"Chi2: {chi2}")
            # print(f"NDF: {ndf}")
            print(f"order {order} Chi2/NDF: {chi2_dof}")
            chi2_dof_l.append(chi2_dof)

            # Extract parameters and their uncertainties
            num_params = fit_func.GetNpar()  # Number of parameters in the fit
            print("Fitted parameters and uncertainties:")
            for i in range(num_params):
                param_value = fit_func.GetParameter(i)  # Fitted parameter value
                param_error = fit_func.GetParError(i)  # Fitted parameter uncertainty
                print(f"order {order}  Parameter {i}: {param_value:} ± {param_error:}")

            canvas = ROOT.TCanvas("canvas", f"{target_nbins} bins SF hist", 800, 600)
            hist_SF.SetLineColor(ROOT.kBlue)
            hist_SF.Draw()
            fit_func.SetLineColor(ROOT.kRed)  # Change color for each fit
            fit_func.Draw("SAME")  # Draw the fit function on the same canvas
            
            
            # Add a legend
            legend = ROOT.TLegend(0.7, 0.7, 0.9, 0.9)  # Legend coordinates (x1, y1, x2, y2)
            legend.AddEntry(hist_SF, "hist_SF", "l")  # "l" means line style
            legend.AddEntry(fit_func, "fit", "l")
            legend.Draw()
            
            # Update the canvas
            canvas.Update()
            
            
            # Save the canvas
            # canvas.SaveAs(f"{save_path}/{year}_njet{njet}_{target_nbins}_order{order}_nbins_SF_fit_byChi2only.png")
            canvas.SaveAs(f"{save_path}/{year}_njet{njet}_{target_nbins}_order{order}_nbins_SF_fit_byChi2only.pdf")
        
        print(f"{target_nbins} minimum chi2 dof: {min(chi2_dof_l)}")
            

2018 njet0------------------------------------------------------------------------------------------------------
rebin_coeff: 10
order 3 Chi2/NDF: 503.1706487834585
Fitted parameters and uncertainties:
order 3  Parameter 0: 0.767726626508058 ± 0.000781512726706322
order 3  Parameter 1: 0.03146666161694277 ± 0.00013360877749381446
order 3  Parameter 2: -0.000741847251796427 ± 4.719787070489866e-06
order 3  Parameter 3: 3.7196878878553673e-06 ± 3.1041693695699896e-08
order 4 Chi2/NDF: 278.8246428619383
Fitted parameters and uncertainties:
order 4  Parameter 0: 0.7234926981811118 ± 0.0008918368599995877
order 4  Parameter 1: 0.04594742872940829 ± 0.00019399998018231906
order 4  Parameter 2: -0.0016515880955840715 ± 1.0018177078153842e-05
order 4  Parameter 3: 1.9837178874466102e-05 ± 1.5960404766514254e-07
order 4  Parameter 4: -6.842587409942315e-08 ± 6.646505758720594e-10
order 5 Chi2/NDF: 95.59631201388105
Fitted parameters and uncertainties:
order 5  Parameter 0: 0.6762488848527612 ± 

Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: pdf file ./plots/2018_njet0_50_order3_nbins_SF_fit_byChi2only.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: pdf file ./plots/2018_njet0_50_order4_nbins_SF_fit_byChi2only.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: pdf file ./plots/2018_njet0_50_order5_nbins_SF_fit_byChi2only.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: pdf file ./plots/2018_njet0_50_order6_nbins_SF_fit_byChi2only.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: pdf file ./plots/201

In [45]:
"""
Try first fitting with Chi2 then by liklihood
"""
year = 2018
# njet = 0
for njet in [0,1,2]:
# for njet in [0]:
    file = ROOT.TFile(f"{year}_njet{njet}.root", "READ")
    save_path = "./plots"
    workspace = file.Get("zpt_Workspace")
    # target_nbins = 50
    # for target_nbins in [25, 50, 100, 250]:
    print(f"{year} njet{njet}------------------------------------------------------------------------------------------------------")
    for target_nbins in [50, 100]:
        hist_data = workspace.obj("hist_data").Clone("hist_data_clone")
        hist_dy = workspace.obj("hist_dy").Clone("hist_dy_clone")
        orig_nbins = hist_data.GetNbinsX()
        rebin_coeff = int(orig_nbins/target_nbins)
        print(f"rebin_coeff: {rebin_coeff}")
        hist_data = hist_data.Rebin(rebin_coeff, "rebinned hist_data") 
        hist_dy = hist_dy.Rebin(rebin_coeff, "rebinned hist_dy") 
        
        hist_SF = hist_data.Clone("hist_SF")
        hist_SF.Divide(hist_dy)
        
        
        
        # Draw the histogram and fit
        canvas = ROOT.TCanvas("canvas", f"{target_nbins} bins Data and DY", 800, 600)
        hist_data.SetLineColor(ROOT.kRed)
        hist_dy.SetLineColor(ROOT.kBlue)
        # Change the plot title
        hist_data.SetTitle(f"njet {njet} {target_nbins} bins Data and DY")
        hist_data.Draw()
        
        hist_dy.Draw("SAME")
        # Add a legend
        legend = ROOT.TLegend(0.7, 0.7, 0.9, 0.9)  # Legend coordinates (x1, y1, x2, y2)
        legend.AddEntry(hist_data, "Data", "l")  # "l" means line style
        legend.AddEntry(hist_dy, "DY", "l")
        legend.Draw()
        canvas.SetLogy(1)
        canvas.Update()
        canvas.SaveAs(f"{save_path}/{year}_njet{njet}_{target_nbins}Bins_DataDy_Hist.png")
        canvas.SaveAs(f"{save_path}/{year}_njet{njet}_{target_nbins}Bins_DataDy_Hist.pdf")
        
        canvas = ROOT.TCanvas("canvas", f"{target_nbins} bins SF hist", 800, 600)
        hist_SF.SetTitle(f"njet {njet} {target_nbins} bins SF")
        # hist_SF.SetMinimum(0.5)
        hist_SF.SetMinimum(-0.25)  # Set the lower bound of the Y-axis
        hist_SF.SetMaximum(4)  # Set the upper bound of the Y-axis
        # hist_SF.SetMaximum(40)  # Set the upper bound of the Y-axis
        hist_SF.Draw()
        
        canvas.Update()
        canvas.SaveAs(f"{save_path}/{year}_njet{njet}_{target_nbins}Bins_SF_Hist.png")
        canvas.SaveAs(f"{save_path}/{year}_njet{njet}_{target_nbins}Bins_SF_Hist.pdf")

        
        dimuon_pt = ROOT.RooRealVar("dimuon_pt", "Dimuon pT", 0, 200)

        # Convert the TH1F histogram to a RooDataHist
        roo_hist_SF = ROOT.RooDataHist("roo_hist", "RooFit Histogram", ROOT.RooArgList(dimuon_pt), hist_SF)
        
        # Print information about the RooDataHist
        roo_hist_SF.Print()
        

        chi2_dof_l = []
        for order in range(3,30):
            hist_SF.SetTitle(f"order {order} njet {njet} {target_nbins} bins SF")
            polynomial_expr = " + ".join([f"[{i}]*x**{i}" for i in range(order+1)])
            # polynomial_func = ROOT.TF1(f"poly{order}", polynomial_expr, -1, 1)
            polynomial_func = ROOT.TF1(f"poly{order}", polynomial_expr, -100, 100)

            # for i in range(num_params):
            #     polynomial_func.SetParLimits(i, -1, 1)         
            
            
            fit_func = polynomial_func
            _ = hist_SF.Fit(fit_func, "L S")
            _ = hist_SF.Fit(fit_func, "L S")
            fit_result = hist_SF.Fit(fit_func, "L S")

            chi2 = fit_func.GetChisquare()
            ndf = fit_func.GetNDF()

            # Calculate chi2/dof
            chi2_dof = chi2 / ndf if ndf > 0 else float("inf")
            
            # Print fit results
            # print(f"Chi2: {chi2}")
            # print(f"NDF: {ndf}")
            print(f"order {order} Chi2/NDF: {chi2_dof}")
            chi2_dof_l.append(chi2_dof)

            # Extract parameters and their uncertainties
            num_params = fit_func.GetNpar()  # Number of parameters in the fit
            print("Fitted parameters and uncertainties:")
            for i in range(num_params):
                param_value = polynomial_func.GetParameter(i)  # Fitted parameter value
                param_error = polynomial_func.GetParError(i)  # Fitted parameter uncertainty
                print(f"order {order}  Parameter {i}: {param_value:} ± {param_error:}")

            canvas = ROOT.TCanvas("canvas", f"{target_nbins} bins SF hist", 800, 600)
            hist_SF.SetLineColor(ROOT.kBlue)
            hist_SF.Draw()
            fit_func.SetLineColor(ROOT.kRed)  # Change color for each fit
            fit_func.Draw("SAME")  # Draw the fit function on the same canvas
            
            
            # Add a legend
            legend = ROOT.TLegend(0.7, 0.7, 0.9, 0.9)  # Legend coordinates (x1, y1, x2, y2)
            legend.AddEntry(hist_SF, "hist_SF", "l")  # "l" means line style
            legend.AddEntry(fit_func, "fit", "l")
            legend.Draw()
            
            # Update the canvas
            canvas.Update()
            
            
            # Save the canvas
            # canvas.SaveAs(f"{save_path}/{year}_njet{njet}_{target_nbins}_order{order}_nbins_SF_fit.png")
            canvas.SaveAs(f"{save_path}/{year}_njet{njet}_{target_nbins}_order{order}_nbins_SF_fit_byLiklihood.png")
        
        print(f"{target_nbins} minimum chi2 dof: {min(chi2_dof_l)}")
            

2018 njet0------------------------------------------------------------------------------------------------------
rebin_coeff: 10
order 3 Chi2/NDF: 0.034486954010837734
Fitted parameters and uncertainties:
order 3  Parameter 0: 0.7896500790745044 ± 0.48977614055723995
order 3  Parameter 1: 0.018382617307315684 ± 0.020370665418496737
order 3  Parameter 2: -0.00020174243381713093 ± 0.00023949363928703045
order 3  Parameter 3: 6.267248801223556e-07 ± 8.053551389545097e-07
order 4 Chi2/NDF: 0.03426325824981995
Fitted parameters and uncertainties:
order 4  Parameter 0: 0.8788962266852722 ± 0.5054231745449437
order 4  Parameter 1: 0.008776836641268538 ± 0.020801947817767317
order 4  Parameter 2: 1.9344239203065406e-05 ± 0.0002888600959815754
order 4  Parameter 3: -1.1137506633300814e-06 ± 2.0424252328937326e-06
order 4  Parameter 4: 4.3891768499459534e-09 ± 5.897466460485503e-09
order 5 Chi2/NDF: 0.034747464733998326
Fitted parameters and uncertainties:
order 5  Parameter 0: 0.935916727363346

Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: png file ./plots/2018_njet0_50Bins_DataDy_Hist.png has been created
Info in <TCanvas::Print>: pdf file ./plots/2018_njet0_50Bins_DataDy_Hist.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: png file ./plots/2018_njet0_50Bins_SF_Hist.png has been created
Info in <TCanvas::Print>: pdf file ./plots/2018_njet0_50Bins_SF_Hist.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: png file ./plots/2018_njet0_50_order3_nbins_SF_fit_byLiklihood.png has been created
Warning in <Fit>: Abnormal termination of minimization.
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: png file ./plots/2018_njet0_50_order4_nbins_SF_fit_byLiklihood.png has been created
Warning in <Fit>: Abnormal termination of minimization.
Warn

In [62]:
"""
likelihood ratio test
"""

year = 2018
# njet = 0
# for njet in [0,1,2]:
for njet in [1,2]:
    file = ROOT.TFile(f"{year}_njet{njet}.root", "READ")
    save_path = "./plots"
    workspace = file.Get("zpt_Workspace")
    # target_nbins = 50
    # for target_nbins in [25, 50, 100, 250]:
    print(f"{year} njet{njet}------------------------------------------------------------------------------------------------------")
    for target_nbins in [50, 100]:
        hist_data = workspace.obj("hist_data").Clone("hist_data_clone")
        hist_dy = workspace.obj("hist_dy").Clone("hist_dy_clone")
        orig_nbins = hist_data.GetNbinsX()
        rebin_coeff = int(orig_nbins/target_nbins)
        print(f"rebin_coeff: {rebin_coeff}")
        hist_data = hist_data.Rebin(rebin_coeff, "rebinned hist_data") 
        hist_dy = hist_dy.Rebin(rebin_coeff, "rebinned hist_dy") 
        
        hist_SF = hist_data.Clone("hist_SF")
        hist_SF.Divide(hist_dy)
        
        
        
        # Draw the histogram and fit
        canvas = ROOT.TCanvas("canvas", f"{target_nbins} bins Data and DY", 800, 600)
        hist_data.SetLineColor(ROOT.kRed)
        hist_dy.SetLineColor(ROOT.kBlue)
        # Change the plot title
        hist_data.SetTitle(f"njet {njet} {target_nbins} bins Data and DY")
        hist_data.Draw()
        
        hist_dy.Draw("SAME")
        # Add a legend
        legend = ROOT.TLegend(0.7, 0.7, 0.9, 0.9)  # Legend coordinates (x1, y1, x2, y2)
        legend.AddEntry(hist_data, "Data", "l")  # "l" means line style
        legend.AddEntry(hist_dy, "DY", "l")
        legend.Draw()
        canvas.SetLogy(1)
        canvas.Update()
        canvas.SaveAs(f"{save_path}/{year}_njet{njet}_{target_nbins}Bins_DataDy_Hist.png")
        canvas.SaveAs(f"{save_path}/{year}_njet{njet}_{target_nbins}Bins_DataDy_Hist.pdf")
        
        canvas = ROOT.TCanvas("canvas", f"{target_nbins} bins SF hist", 800, 600)
        hist_SF.SetTitle(f"njet {njet} {target_nbins} bins SF")
        hist_SF.SetMinimum(0.5)  # Set the lower bound of the Y-axis
        hist_SF.SetMaximum(4)  # Set the upper bound of the Y-axis
        hist_SF.Draw()
        
        canvas.Update()
        canvas.SaveAs(f"{save_path}/{year}_njet{njet}_{target_nbins}Bins_SF_Hist.png")
        canvas.SaveAs(f"{save_path}/{year}_njet{njet}_{target_nbins}Bins_SF_Hist.pdf")

        
        dimuon_pt = ROOT.RooRealVar("dimuon_pt", "Dimuon pT", 0, 200)

        # Convert the TH1F histogram to a RooDataHist
        roo_hist_SF = ROOT.RooDataHist("roo_hist", "RooFit Histogram", ROOT.RooArgList(dimuon_pt), hist_SF)
        
        # Print information about the RooDataHist
        roo_hist_SF.Print()
        


        for order in range(3,30):
            # Define two polynomial orders
            order_low = order
            order_high = order + 1
                
            # Fit with the lower-order polynomial
            polynomial_expr = " + ".join([f"[{i}]*x**{i}" for i in range(order_low + 1)])
            polynomial_func = ROOT.TF1(f"poly{order}", polynomial_expr, -5, 5)
            # Define the TF1 function with the generated expression
            fit_func_low = polynomial_func
            _ = hist_SF.Fit(fit_func_low, "S")
            _ = hist_SF.Fit(fit_func_low, "S")
            fit_low = hist_SF.Fit(fit_func_low, "L S")
            low_nll = fit_low.MinFcnValue() # https://root-forum.cern.ch/t/likelihood-for-evaluating-goodness-of-fit/28156
            # print(f"low_nll: {low_nll}")

            
            chi2_low = fit_func_low.GetChisquare()
            ndf_low = fit_func_low.GetNDF()
            # log_likelihood_low = fit_func_low.GetLogLikelihood()
            # print(f"log_likelihood_low: {log_likelihood_low}")
            
            # Fit with the higher-order polynomial
            polynomial_expr = " + ".join([f"[{i}]*x**{i}" for i in range(order_high + 1)])
            # Define the TF1 function with the generated expression
            polynomial_func = ROOT.TF1(f"poly{order}", polynomial_expr, -5, 5)
            # Define the TF1 function with the generated expression
            fit_func_high = polynomial_func
            _ = hist_SF.Fit(fit_func_high, "S")
            _ = hist_SF.Fit(fit_func_high, "S")
            fit_high = hist_SF.Fit(fit_func_high, "L S")
            high_nll = fit_high.MinFcnValue() # https://root-forum.cern.ch/t/likelihood-for-evaluating-goodness-of-fit/28156
            
            chi2_high = fit_func_high.GetChisquare()
            ndf_high = fit_func_high.GetNDF()
            
            
            # Calculate F-statistic
            delta_chi2 = chi2_low - chi2_high
            delta_dof = ndf_high - ndf_low
            # f_statistic = (delta_chi2 / delta_dof) / (chi2_high / ndf_high)

            delta_nll = 2*(low_nll-high_nll) # line 1552 if AN-19-124
            delta_order = order_high - order_low
            p_value = ROOT.TMath.Prob(delta_nll, delta_order)
            
            # Print results
            print(f"Lower-order {target_nbins} bins polynomial (pol{order_low}): chi2 = {chi2_low}, ndf = {ndf_low}, chi2_dof = {chi2_low/ndf_low}")
            print(f"Higher-order {target_nbins} bins polynomial (pol{order_high}): chi2 = {chi2_high}, ndf = {ndf_high}, chi2_dof = {chi2_high/ndf_high}")
            print(f"delta_nll {target_nbins} bins: {delta_nll}")
            print(f"P-value {target_nbins} bins: {p_value}")
            
            if p_value < 0.05:  # Typically, p-value < 0.05 indicates significant improvement
                print(f"Higher-order {order_high} polynomial significantly improves the fit.")
            else:
                print(f"Higher-order {order_high} polynomial does not significantly improve the fit.")

2018 njet1------------------------------------------------------------------------------------------------------
rebin_coeff: 10
Lower-order 50 bins polynomial (pol3): chi2 = 0.17399466221937143, ndf = 46, chi2_dof = 0.003782492656942857
Higher-order 50 bins polynomial (pol4): chi2 = 0.1712804731774803, ndf = 45, chi2_dof = 0.00380623273727734
delta_nll 50 bins: 0.002714189041891135
P-value 50 bins: 0.9584507026370993
Higher-order 4 polynomial does not significantly improve the fit.
Lower-order 50 bins polynomial (pol4): chi2 = 0.1712804731774803, ndf = 45, chi2_dof = 0.00380623273727734
Higher-order 50 bins polynomial (pol5): chi2 = 0.16485649606427893, ndf = 44, chi2_dof = 0.0037467385469154304
delta_nll 50 bins: 0.006423977113201362
P-value 50 bins: 0.9361181815628861
Higher-order 5 polynomial does not significantly improve the fit.
Lower-order 50 bins polynomial (pol5): chi2 = 0.16485649606427893, ndf = 44, chi2_dof = 0.0037467385469154304
Higher-order 50 bins polynomial (pol6): ch

Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: png file ./plots/2018_njet1_50Bins_DataDy_Hist.png has been created
Info in <TCanvas::Print>: pdf file ./plots/2018_njet1_50Bins_DataDy_Hist.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: png file ./plots/2018_njet1_50Bins_SF_Hist.png has been created
Info in <TCanvas::Print>: pdf file ./plots/2018_njet1_50Bins_SF_Hist.pdf has been created
Warning in <Fit>: Abnormal termination of minimization.
Warning in <Fit>: Abnormal termination of minimization.
Warning in <Fit>: Abnormal termination of minimization.
Warning in <Fit>: Abnormal termination of minimization.
Warning in <Fit>: Abnormal termination of minimization.
Warning in <Fit>: Abnormal termination of minimization.
Warning in <Fit>: Abnormal termination of minimization.
Warning in <Fit>: Abnormal termination of minimization.
Warning in <Fit>: Abnormal terminati

In [67]:
"""
F-test
"""


import ROOT
from scipy.stats import f
# year = 2017
# year = "2016postVFP"
year = "2018"
# njet = 0
for njet in [0,1,2]:
# for njet in [1,2]:
    file = ROOT.TFile(f"{year}_njet{njet}.root", "READ")
    save_path = "./plots"
    workspace = file.Get("zpt_Workspace")
    # target_nbins = 50
    # for target_nbins in [25, 50, 100, 250]:
    print(f"{year} njet{njet}------------------------------------------------------------------------------------------------------")
    for target_nbins in [50, 100]:
        hist_data = workspace.obj("hist_data").Clone("hist_data_clone")
        hist_dy = workspace.obj("hist_dy").Clone("hist_dy_clone")
        orig_nbins = hist_data.GetNbinsX()
        rebin_coeff = int(orig_nbins/target_nbins)
        print(f"rebin_coeff: {rebin_coeff}")
        hist_data = hist_data.Rebin(rebin_coeff, "rebinned hist_data") 
        hist_dy = hist_dy.Rebin(rebin_coeff, "rebinned hist_dy") 
        
        hist_SF = hist_data.Clone("hist_SF")
        hist_SF.Divide(hist_dy)
        
        
        
        # Draw the histogram and fit
        canvas = ROOT.TCanvas("canvas", f"{target_nbins} bins Data and DY", 800, 600)
        hist_data.SetLineColor(ROOT.kRed)
        hist_dy.SetLineColor(ROOT.kBlue)
        # Change the plot title
        hist_data.SetTitle(f"njet {njet} {target_nbins} bins Data and DY")
        hist_data.Draw()
        
        hist_dy.Draw("SAME")
        # Add a legend
        legend = ROOT.TLegend(0.7, 0.7, 0.9, 0.9)  # Legend coordinates (x1, y1, x2, y2)
        legend.AddEntry(hist_data, "Data", "l")  # "l" means line style
        legend.AddEntry(hist_dy, "DY", "l")
        legend.Draw()
        canvas.SetLogy(1)
        canvas.Update()
        canvas.SaveAs(f"{save_path}/{year}_njet{njet}_{target_nbins}Bins_DataDy_Hist.png")
        canvas.SaveAs(f"{save_path}/{year}_njet{njet}_{target_nbins}Bins_DataDy_Hist.pdf")
        
        canvas = ROOT.TCanvas("canvas", f"{target_nbins} bins SF hist", 800, 600)
        hist_SF.SetTitle(f"njet {njet} {target_nbins} bins SF")
        hist_SF.SetMinimum(0.5)  # Set the lower bound of the Y-axis
        hist_SF.SetMaximum(4)  # Set the upper bound of the Y-axis
        hist_SF.Draw()
        
        canvas.Update()
        canvas.SaveAs(f"{save_path}/{year}_njet{njet}_{target_nbins}Bins_SF_Hist.png")
        canvas.SaveAs(f"{save_path}/{year}_njet{njet}_{target_nbins}Bins_SF_Hist.pdf")

        
        dimuon_pt = ROOT.RooRealVar("dimuon_pt", "Dimuon pT", 0, 200)

        # Convert the TH1F histogram to a RooDataHist
        roo_hist_SF = ROOT.RooDataHist("roo_hist", "RooFit Histogram", ROOT.RooArgList(dimuon_pt), hist_SF)
        
        # Print information about the RooDataHist
        roo_hist_SF.Print()
        


        for order in range(3,16):
            # Define two polynomial orders
            order_low = order
            order_high = order + 1
                
            # Fit with the lower-order polynomial
            polynomial_expr = " + ".join([f"[{i}]*x**{i}" for i in range(order_low + 1)])
            polynomial_func = ROOT.TF1(f"poly{order}", polynomial_expr, -5, 5)
            # Define the TF1 function with the generated expression
            fit_func_low = polynomial_func
            _ = hist_SF.Fit(fit_func_low, "S")
            _ = hist_SF.Fit(fit_func_low, "S")
            fit_low = hist_SF.Fit(fit_func_low, "S")

            
            chi2_low = fit_func_low.GetChisquare()
            ndf_low = fit_func_low.GetNDF()
            # log_likelihood_low = fit_func_low.GetLogLikelihood()
            # print(f"log_likelihood_low: {log_likelihood_low}")
            
            # Fit with the higher-order polynomial
            polynomial_expr = " + ".join([f"[{i}]*x**{i}" for i in range(order_high + 1)])
            # Define the TF1 function with the generated expression
            polynomial_func = ROOT.TF1(f"poly{order}", polynomial_expr, -5, 5)
            # Define the TF1 function with the generated expression
            fit_func_high = polynomial_func
            _ = hist_SF.Fit(fit_func_high, "S")
            _ = hist_SF.Fit(fit_func_high, "S")
            fit_high = hist_SF.Fit(fit_func_high, "S")
            
            chi2_high = fit_func_high.GetChisquare()
            ndf_high = fit_func_high.GetNDF()
            
            
            # Calculate F-statistic
            delta_chi2 = chi2_low - chi2_high
            delta_dof = -(ndf_high - ndf_low)
            f_statistic = delta_chi2 / chi2_high * (ndf_high) / delta_dof
            print(f"(target_nbins - order_high): {(target_nbins - order_high)}")
            print(f"ndf_high: {ndf_high}")
            print(f"delta_dof: {delta_dof}")
            print(f"f_statistic: {f_statistic}")
            # Calculate the p-value (use scipy.stats.f for F-distribution)
            # p_value = 1 - f.cdf(f_statistic, delta_dof, ndf_high)
            p_value = 1 - f.cdf(f_statistic, delta_dof, ndf_high)
            # p_value = ROOT.TMath.Prob(f_statistic, delta_dof)
            # delta_nll = 2*(low_nll-high_nll) # line 1552 if AN-19-124
            

            
            # Print results
            print(f"Lower-order {target_nbins} bins polynomial (pol{order_low}): chi2 = {chi2_low}, ndf = {ndf_low}, chi2_dof = {chi2_low/ndf_low}")
            print(f"Higher-order {target_nbins} bins polynomial (pol{order_high}): chi2 = {chi2_high}, ndf = {ndf_high}, chi2_dof = {chi2_high/ndf_high}")
            print(f"F-statistic {target_nbins} bins: {f_statistic}")
            print(f"P-value {target_nbins} bins: {p_value}")
            
            if p_value < 0.05:  # Typically, p-value < 0.05 indicates significant improvement
                print(f"Higher-order {order_high} polynomial significantly improves the fit. chi2_low: {chi2_low/ndf_low} vs chi2_high: {chi2_high/ndf_high}")
            else:
                print(f"Higher-order {order_high} polynomial does not significantly improve the fit.")

2018 njet0------------------------------------------------------------------------------------------------------
rebin_coeff: 10
(target_nbins - order_high): 46
ndf_high: 45
delta_dof: 1
f_statistic: 1.3001420208276695
Lower-order 50 bins polynomial (pol3): chi2 = 1.5864022825328308, ndf = 46, chi2_dof = 0.03448700614201806
Higher-order 50 bins polynomial (pol4): chi2 = 1.541854940355564, ndf = 45, chi2_dof = 0.034263443119012534
F-statistic 50 bins: 1.3001420208276695
P-value 50 bins: 0.26021989242682153
Higher-order 4 polynomial does not significantly improve the fit.
(target_nbins - order_high): 45
ndf_high: 44
delta_dof: 1
f_statistic: 0.3731463813003434
Lower-order 50 bins polynomial (pol4): chi2 = 1.541854940355564, ndf = 45, chi2_dof = 0.034263443119012534
Higher-order 50 bins polynomial (pol5): chi2 = 1.5288890445739163, ndf = 44, chi2_dof = 0.03474747828577082
F-statistic 50 bins: 0.3731463813003434
P-value 50 bins: 0.5444370576018099
Higher-order 5 polynomial does not signifi

Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: png file ./plots/2018_njet0_50Bins_DataDy_Hist.png has been created
Info in <TCanvas::Print>: pdf file ./plots/2018_njet0_50Bins_DataDy_Hist.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: png file ./plots/2018_njet0_50Bins_SF_Hist.png has been created
Info in <TCanvas::Print>: pdf file ./plots/2018_njet0_50Bins_SF_Hist.pdf has been created
Warning in <Fit>: Abnormal termination of minimization.
Warning in <Fit>: Abnormal termination of minimization.
Warning in <Fit>: Abnormal termination of minimization.
Warning in <Fit>: Abnormal termination of minimization.
Warning in <Fit>: Abnormal termination of minimization.
Warning in <Fit>: Abnormal termination of minimization.
Warning in <Fit>: Abnormal termination of minimization.
Warning in <Fit>: Abnormal termination of minimization.
Warning in <Fit>: Abnormal terminati

In [69]:
# Draw the histogram and fit

order = 7

canvas = ROOT.TCanvas("canvas", f"{order}th-Order Polynomial Fit", 800, 600)


# Define the order of the polynomial

# fit_func = hist_SF.GetFunction(f"pol{order}")

# Generate the polynomial expression dynamically
polynomial_expr = " + ".join([f"[{i}]*x**{i}" for i in range(order + 1)])

# Define the TF1 function with the generated expression
polynomial_func = ROOT.TF1(f"poly{order}", polynomial_expr, -5, 5)

# Set initial parameter guesses (optional)
for i in range(order + 1):
    polynomial_func.SetParameter(i, 1)  # Initial guess for each parameter

fit_func = polynomial_func
# print(fit_func)

_ = hist_SF.Fit(fit_func, "S")  # "S" option ensures we get the fit result
fit_result = hist_SF.Fit(fit_func, "S")  # "S" option ensures we get the fit result
# print(fit_result)
# fit_result.Print()
# Extract chi2 and dof
# fit_func = hist_SF.GetFunction("pol3")
chi2 = fit_func.GetChisquare()  # Total chi-squared
ndf = fit_func.GetNDF()  # Number of degrees of freedom

# Calculate chi2/dof
chi2_dof = chi2 / ndf if ndf > 0 else float("inf")

# Print fit results
print(f"Chi2: {chi2}")
print(f"NDF: {ndf}")
print(f"Chi2/NDF: {chi2_dof}")

# Extract parameters and their uncertainties
num_params = fit_func.GetNpar()  # Number of parameters in the fit
print("Fitted parameters and uncertainties:")
for i in range(num_params):
    param_value = fit_func.GetParameter(i)  # Fitted parameter value
    param_error = fit_func.GetParError(i)  # Fitted parameter uncertainty
    print(f"  Parameter {i}: {param_value:.4f} ± {param_error:.4f}")

hist_SF.SetLineColor(ROOT.kBlue)
hist_SF.Draw()
fit_func.SetLineColor(ROOT.kRed)  # Change color for each fit
fit_func.Draw("SAME")  # Draw the fit function on the same canvas


# Add a legend
legend = ROOT.TLegend(0.7, 0.7, 0.9, 0.9)  # Legend coordinates (x1, y1, x2, y2)
legend.AddEntry(hist_SF, "hist_SF", "l")  # "l" means line style
legend.AddEntry(fit_func, "fit", "l")
legend.Draw()

# Update the canvas
canvas.Update()


# Save the canvas
canvas.SaveAs(f"{save_path}/{year}_njet{njet}_SF_fit.png")


Chi2: 1170.9752966120684
NDF: 42
Chi2/NDF: 27.880364205049247
****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      1170.98
NDf                       =           42
Edm                       =  1.93314e-09
NCalls                    =          741
p0                        =     0.635804   +/-   0.000745401 
p1                        =    0.0844285   +/-   0.000132114 
p2                        =  -0.00548066   +/-   5.98055e-06 
p3                        =  0.000157687   +/-   1.00132e-07 
p4                        = -2.20465e-06   +/-   7.8029e-10  
p5                        =  1.57946e-08   +/-   5.08576e-12 
p6                        = -5.59425e-11   +/-   2.9218e-14  
p7                        =  7.77105e-14   +/-   1.20302e-16 


Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: png file test_polynomial_fits.png has been created


In [112]:
import ROOT
import math

# Create a histogram
hist = ROOT.TH1D("hist", "Example Histogram", 50, -5, 5)

# Fill the histogram with Gaussian-distributed random data
for _ in range(1000):
    hist.Fill(ROOT.gRandom.Gaus(0, 1))

# Fit the histogram with a Gaussian function
fit_result = hist.Fit("gaus", "S")  # "S" option ensures the fit result is accessible
fit_func = hist.GetFunction("gaus")  # Get the fitted function

if fit_func:
    # Extract the log-likelihood value
    log_likelihood = fit_func.GetLogLikelihood()

    # Calculate the likelihood
    likelihood = math.exp(log_likelihood)

    # Print the results
    print(f"Log-likelihood value: {log_likelihood}")
    print(f"Likelihood value: {likelihood}")
else:
    print("Fit function not available.")


AttributeError: 'TF1' object has no attribute 'GetLogLikelihood'

****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      34.7164
NDf                       =           29
Edm                       =  9.72069e-09
NCalls                    =           59
Constant                  =      74.9237   +/-   3.00474     
Mean                      =    0.0111669   +/-   0.0336562   
Sigma                     =      1.03102   +/-   0.0253614    	 (limited)


In [91]:
from omegaconf import OmegaConf
import awkward as ak


def getZptWgts(dimuon_pt, njets, nbins, year):
    config_path = "zpt_rewgt_params.yaml"
    wgt_config = OmegaConf.load(config_path)
    max_order = 9
    zpt_wgt = ak.ones_like(dimuon_pt)
    jet_multiplicies = [0,1,2]
    print(f"zpt_wgt: {zpt_wgt}")
    
    for jet_multiplicity in jet_multiplicies:
        zpt_wgt_by_jet = ak.zeros_like(dimuon_pt)
        for order in range(max_order+1): # p goes from 0 to max_order
            coeff = wgt_config[str(year)][f"njet_{jet_multiplicity}"][nbins][f"p{order}"]
            print(f"njet{jet_multiplicity} order {order} coeff: {coeff}")
            polynomial_term = coeff*dimuon_pt**order
            zpt_wgt_by_jet = zpt_wgt_by_jet + polynomial_term
            print(f"njet{jet_multiplicity} order {order} polynomial_term: {polynomial_term}")
            print(f"njet{jet_multiplicity} order {order} zpt_wgt_by_jet: {zpt_wgt_by_jet}")

        if jet_multiplicity != 2:
            njet_mask = njets == jet_multiplicity
        else:
            njet_mask = njets >= 2 # njet 2 is inclusive
        print(f"njet{jet_multiplicity} order  zpt_wgt_by_jet: {zpt_wgt_by_jet}")
        zpt_wgt = ak.where(njet_mask, zpt_wgt_by_jet, zpt_wgt) # if matching jet multiplicity, apply the values
        print(f"zpt_wgt after njet {jet_multiplicity}: {zpt_wgt}")
    cutOff_mask = dimuon_pt < 200 # ignore wgts from dimuon pT > 200
    zpt_wgt = ak.where(cutOff_mask, zpt_wgt, ak.ones_like(dimuon_pt))
    print(f"final zpt_wgt: {zpt_wgt}")
    return zpt_wgt

In [92]:
dimuon_pt = ak.Array([1,2,3,4,5,400])
njets = ak.Array([1,2,0,2,9,0])
year = 2018
nbins = 50
getZptWgts(dimuon_pt, njets, nbins, year)

zpt_wgt: [1, 1, 1, 1, 1, 1]
njet0 order 0 coeff: 0.6566487226684253
njet0 order 0 polynomial_term: [0.657, 0.657, 0.657, 0.657, 0.657, 0.657]
njet0 order 0 zpt_wgt_by_jet: [0.657, 0.657, 0.657, 0.657, 0.657, 0.657]
njet0 order 1 coeff: 0.07078938894708699
njet0 order 1 polynomial_term: [0.0708, 0.142, 0.212, 0.283, 0.354, 28.3]
njet0 order 1 zpt_wgt_by_jet: [0.727, 0.798, 0.869, 0.94, 1.01, 29]
njet0 order 2 coeff: -0.0030950655289551997
njet0 order 2 polynomial_term: [-0.0031, -0.0124, -0.0279, -0.0495, -0.0774, -495]
njet0 order 2 zpt_wgt_by_jet: [0.724, 0.786, 0.841, 0.89, 0.933, -466]
njet0 order 3 coeff: -1.3854135159104706e-05
njet0 order 3 polynomial_term: [-1.39e-05, -0.000111, -0.000374, -0.000887, -0.00173, -887]
njet0 order 3 zpt_wgt_by_jet: [0.724, 0.786, 0.841, 0.889, 0.931, -1.35e+03]
njet0 order 4 coeff: 3.890530132026851e-06
njet0 order 4 polynomial_term: [3.89e-06, 6.22e-05, 0.000315, 0.000996, 0.00243, 9.96e+04]
njet0 order 4 zpt_wgt_by_jet: [0.724, 0.786, 0.841, 0.89

<Array [0.719, 0.778, 0.841, 0.872, 0.911, 1] type='6 * float64'>